In [3]:
import numpy as np
from Music_Style_Transfer_master.project.midi_handler import score2midi, midi2score
from Music_Style_Transfer_master.project.utils import add_beat, padding, load_model, get_representation, dataset_import
from Music_Style_Transfer_master.project.test import style_transfer
from Music_Style_Transfer_master.project.model_transformer_relative import _2way_transformer_wavenet_absolute_pitch, _2way_transformer_wavenet_relative_pitch

import tqdm
import pypianoroll as pr
import tensorflow as tf
import os

import matplotlib.pyplot as plt
#from keras.models import load_model
%matplotlib inline

# 33:97 A1~C7  21:109 A0~C8
# 7 octave 21:105  A0~G#7  
def midi2score(path, beat_resolution, pitch_range=np.arange(21, 109)):
    tmp = pr.Multitrack(filename=path, beat_resolution=beat_resolution)   
    tmp.binarize()
    tmp.assign_constant(1)
    # TODO: use variable to handle pitch range
    score = tmp.get_merged_pianoroll()[:, pitch_range]
    
    return score

def load_model(path_model):
    model = _2way_transformer_wavenet_absolute_pitch(len_context = 8*4*beat_resolution,
                                                 n_octave = 7,
                                                 size_embedding = 84,
                                                 n_transf_layers = 4,
                                                 n_transf_downsample = 0,
                                                 n_conv_layers = 7,
                                                 context_layers = 1)
    model.load_weights(path_model)
    
    return model

def time_index_chosing(_range,
                       interval,
                       random=True,
                       time_list=None):
    # For parallel generation
    
    interval += 1
    if (random):
        time_index_base = np.random.randint(_range)
    else:
        time_index_base = time_list[np.random.randint(len(time_list))]

    c = time_index_base + np.arange(-interval * 200, interval * 200, interval)

    return c[np.where(np.logical_and(c >= 0, c < _range))]


def midi_index_chosing(_len, pitch_range):
    return np.random.randint(pitch_range, size=_len)


def generation(model, score, meta, time_indexes, midi_indexes,
                       len_context
                       ):
    time_indexes = np.array(time_indexes) +  len_context
    
    left_features = (np.array(score[[np.arange(t -  len_context, t) for t in time_indexes], :]) > 0).astype(int)
    left_metas = np.array(meta[[np.arange(t -  len_context, t) for t in time_indexes], :])
    
    right_features = (np.array(score[[np.arange(t + 1, t + 1 + len_context) for t in time_indexes], :]) > 0).astype(int)
    right_metas = np.array(meta[[np.arange(t + 1, t + 1 + len_context) for t in time_indexes], :])
    
    central_features = (np.array(score[time_indexes, :]) > 0).astype(int)
    central_metas = np.array(meta[time_indexes, :])
    
    for a, b in enumerate(midi_indexes):
        central_features[a, b:] = 2
        
    central_features = np.reshape(central_features, (len(time_indexes), 1, -1))
    central_metas = np.reshape(central_metas, (len(time_indexes), 1, -1))

    p = model.predict([left_features,  left_metas, 
                       central_features, central_metas,
                       right_features, right_metas])
    
    return p

def style_transfer_new(score, meta, score_melody, model, len_context, pitch_range, 
                   iter_num=25, threshold=0.5
                   ):

    fixed_rhythm_score = score_melody
    original_len = len(score_melody)
    new_score = np.array(score)
    counter = 0
    alpha_initial = 0.6
    alpha = alpha_initial
    alpha_min = 0
    annealing_fraction = 0.6
    update_count = 0
    
    for i in tqdm.tqdm(range(iter_num)):
        time_list = np.arange(original_len)
        if i < iter_num*0.9:
            score[len_context:-len_context] = np.logical_or(score[len_context:-len_context], score_melody).astype(int)
        print("alpha = ", alpha)
        
        while (time_list.size > 0):
            if(alpha != -1):
                alpha = max(0, alpha_initial - update_count * (alpha_initial - alpha_min) / (
                    iter_num * original_len * annealing_fraction))
            if(alpha == 0):
                score = new_score
                alpha = -1
            elif(counter / original_len > alpha and alpha != -1):
                counter = 0
                score = np.array(new_score)

            time_indexes = time_index_chosing(original_len, len_context, random=False, time_list=time_list)
            l = len(time_indexes)
            sorter = np.argsort(time_list)
            d = sorter[np.searchsorted(time_list, time_indexes, sorter=sorter)]
            time_list = np.delete(time_list, d, 0)
            counter += l

            update_count += l

            if(alpha != -1):
                midi_indexes = np.arange(pitch_range).tolist() * len(time_indexes)
                time_indexes_repeat = np.repeat(time_indexes, pitch_range)
                p = generation(model, score, meta, time_indexes_repeat, midi_indexes, len_context)
                
                # TODO: This for loop seems can be removed
                for j, t in enumerate(time_indexes_repeat):
                    update = p[0][j]
#                    if(fixed_rhythm_score[t, midi_indexes[j]] == 0):
#                     new_score[t + len_context, midi_indexes[j]] = np.random.binomial(1, p[0][j][1])
                    if update < threshold :
                        new_score[t + len_context, midi_indexes[j]] = 0
                    else:
                        new_score[t + len_context, midi_indexes[j]] = 1
               
            else:
                for midi_index in range(pitch_range):
                    
                    midi_indexes = [midi_index] * l
                    p = generation(model, score, meta, time_indexes, midi_indexes, len_context)
                        
                    for j, t in enumerate(time_indexes):
                        update = p[0][j]
                        if i > (0.8 *iter_num):
                            if(fixed_rhythm_score[t, midi_indexes[j]] == 0):
                                if update < threshold :
                                    new_score[t + len_context, midi_indexes[j]] = 0
                                else:
                                    new_score[t + len_context, midi_indexes[j]] = 1
                        else:   
#                         if(fixed_rhythm_score[t, midi_indexes[j]] == 0):
#                             new_score[t + len_context, midi_indexes[j]] = np.random.binomial(1, p[0][j][1])
                            if update < threshold :
                                new_score[t + len_context, midi_indexes[j]] = 0
                            else:
                                new_score[t + len_context, midi_indexes[j]] = 1
        
    return new_score[len_context:-len_context, :pitch_range]

In [2]:
beat_resolution = 8
pitch_range = 84
len_context = beat_resolution*4*8
max_len = beat_resolution*4*4

path_model = "../../../scratch/wtl272/projects/Music_style_transfer/runs/run_53/model/model.hdf5"
model = load_model(path_model)

In [4]:
path = "input_midi/0716/"
file_list = dataset_import(path, ".mid")

for i in file_list:
    if "harmonized" in i :
        file_list.remove(i)

Load from input_midi/0716/
22 files loaded!


In [5]:
file_list

['input_midi/0716/bicycle.mid',
 'input_midi/0716/donkey.mid',
 'input_midi/0716/happy_birthday.mid',
 'input_midi/0716/home.mid',
 'input_midi/0716/indians.mid',
 'input_midi/0716/londonbridge.mid',
 'input_midi/0716/macdonald.mid',
 'input_midi/0716/stars.mid',
 'input_midi/0716/together.mid',
 'input_midi/0716/train.mid',
 'input_midi/0716/two_tigers.mid']

In [6]:
for i in file_list:

    path = i[:-4] + "_harmonized.mid"
    path_melody = i
    print(path, path_melody)
    
    score = midi2score(path, beat_resolution, pitch_range=np.arange(21, 105))
    
    if path_melody is None:
        score_melody = np.zeros(score.shape)
    else:
        score_melody = midi2score(path_melody, beat_resolution, pitch_range=np.arange(21, 105))

    # New version
    score = np.array(score[:])
    score_melody = np.array(score_melody[:])
    score, score_meta = get_representation(score, pitch_range, beat_resolution, len_context)

    #generation
    result = style_transfer_new(score, score_meta, score_melody, model, len_context, pitch_range, iter_num=15)

    #save result
    score2midi("output_midi/Results_1112/" + os.path.basename(i).split(".")[0] + "_bach_model53.mid", 
               result[:len(score_melody)], beat_resolution, 120, 
               pitch_range=np.arange(21, 105),melody_constraint=True, melody=score_melody)


  0%|          | 0/15 [00:00<?, ?it/s]

input_midi/0716/bicycle_harmonized.mid input_midi/0716/bicycle.mid
alpha =  0.6


  7%|▋         | 1/15 [00:25<05:59, 25.67s/it]

alpha =  0.5338235294117647


 13%|█▎        | 2/15 [00:42<04:57, 22.91s/it]

...

 80%|████████  | 12/15 [32:43<17:28, 349.39s/it]

alpha =  -1


 87%|████████▋ | 13/15 [41:09<13:12, 396.37s/it]

alpha =  -1


KeyboardInterrupt: 

In [ ]:
for i in file_list:

    path = i[:-4] + "_harmonized.mid"
    path_melody = i
    print(path, path_melody)
    
    score = midi2score(path, beat_resolution, pitch_range=np.arange(21, 105))
    
    if path_melody is None:
        score_melody = np.zeros(score.shape)
    else:
        score_melody = midi2score(path_melody, beat_resolution, pitch_range=np.arange(21, 105))

    # New version
    score = np.array(score[:])
    score_melody = np.array(score_melody[:])
    score, score_meta = get_representation(score, pitch_range, beat_resolution, len_context)

    #generation
    result = style_transfer_new(score, score_meta, score_melody, model, len_context, pitch_range, iter_num=15)

    #save result
    score2midi("output_midi/Results_1022/" + os.path.basename(i).split(".")[0] + "_bach_model49.mid", 
               result[:len(score_melody)], beat_resolution, 120, 
               pitch_range=np.arange(21, 105),melody_constraint=True, melody=score_melody)


## def len_generator(path_dataset, len_context, phase="valid", percentage_train=0.8,):
    d_scores = dataset_import(path_dataset, ".npy")
    d_metas = []
    for i in d_scores:
        if "meta" in i:
            d_metas.append(i)
            d_scores.remove(i)
         
    d_metas.sort(key=lambda x: (int(os.path.basename(x)[:-9])))
    
    # Train valid test split:  percentage_train,  0.5*(1-percentage_train), 0.5*(1-percentage_train)
    if phase == 'train':
        score_indices = np.arange(int(len(d_scores) * percentage_train))
    if phase == "valid":
        score_indices = np.arange(int(len(d_scores) * percentage_train), int((1 + percentage_train)/2 * len(d_scores)))
    elif phase == 'test':
        score_indices = np.arange(int((1 + percentage_train)/2 * len(d_scores)), len(d_scores))
    elif phase == 'all':
        score_indices = np.arange(len(d_scores))

    num_samples = 0
    for score_index in score_indices:
        score = np.load(d_scores[score_index])
        num_samples += (score.shape[0] - 2*len_context)*score.shape[1]    
    
    return num_samples

In [21]:
p = "../../../scratch/wtl272/dataset/bach_8_on_offset/"
len_generator(p, 256)/84

Load from ../../../scratch/wtl272/dataset/bach_8_on_offset/
742 files loaded!


16832.0